 # Segmenting and Clustering Neighborhoods in Toronto

Fabian

In [25]:
# importing
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
#!pip install geocoder
import geocoder
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

print("Environment ready")

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  24.43 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  11.90 MB/s
Environment ready


# 1. Part 

## Read data from wikipedia

In [2]:
#Read data from wikipedia
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#read raw text
raw_text = requests.get(link).text

#parse text as xml
soup = BeautifulSoup(raw_text,'xml')



## Clean data

In [3]:
#extract table
table = soup.find('table')

#create empty list
Postcode      = []
Borough       = []
Neighbourhood = []

#<tr>
#<td>M5A</td>
#<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
#<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
#</td></tr>
#<tr>

#each line has <tr> as keyword
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)
    


In [4]:
#three lists
print(Postcode)
print(Borough)
print(Neighbourhood)

['M3A', 'M4A', 'M5A', 'M5A', 'M6A', 'M6A', 'M9A', 'M1B', 'M1B', 'M4B', 'M4B', 'M5B', 'M6B', 'M9B', 'M9B', 'M9B', 'M1C', 'M1C', 'M1C', 'M3C', 'M4C', 'M5C', 'M9C', 'M1E', 'M1E', 'M4E', 'M5E', 'M1G', 'M4G', 'M1H', 'M2H', 'M3H', 'M3H', 'M4H', 'M5H', 'M5H', 'M6H', 'M1J', 'M2J', 'M3J', 'M3J', 'M4J', 'M5J', 'M5J', 'M6J', 'M6J', 'M1K', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M5K', 'M6K', 'M6K', 'M1L', 'M1L', 'M1L', 'M2L', 'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M9L', 'M1M', 'M1M', 'M2M', 'M2M', 'M5M', 'M6M', 'M6M', 'M6M', 'M9M', 'M9M', 'M1N', 'M4N', 'M9N', 'M1P', 'M1P', 'M1P', 'M5P', 'M6P', 'M9P', 'M1R', 'M1R', 'M2R', 'M5R', 'M5R', 'M6R', 'M6R', 'M9R', 'M1S', 'M5S', 'M6S', 'M6S', 'M1T', 'M4T', 'M5T', 'M5T', 'M5T', 'M1V', 'M1V', 'M4V', 'M4V', 'M4V', 'M5V', 'M5V', 'M5V', 'M5V', 'M8V', 'M9V', 'M9V', 'M9V', 'M9V', 'M9V', 'M9V', 'M1W', 'M4W', 'M8W', 'M8W', 'M9W', 'M1X', 'M4X', 'M4X', 'M5X', 'M5X', 'M8X', 'M4Y', 'M8Y', 'M8Y', 'M8Y', 'M8Y', 'M8Y', 'M8Y', 'M8Y', 'M8Z', 'M8Z', 'M8Z']
['North York', 'North York

## more neighbourhoods for one postcode

In [5]:
#more neighbourhoods for one postcode
unique_p = set(Postcode)

Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

In [96]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto = df_toronto[['Postcode', 'Borough', 'Neighbourhood']]
df_toronto = df_toronto.sort_values('Postcode')
df_toronto.head(20)

,Postcode,Borough,Neighbourhood
32,M1B,Scarborough,"Rouge, Malvern"
72,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
65,M1E,Scarborough,"Morningside, West Hill"
75,M1G,Scarborough,Woburn
41,M1H,Scarborough,Cedarbrae
24,M1J,Scarborough,Scarborough Village
37,M1K,Scarborough,"Ionview, Kennedy Park"
18,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
60,M1M,Scarborough,"Cliffcrest, Cliffside"
11,M1N,Scarborough,Birch Cliff


In [8]:
df_toronto.shape

(84, 3)

## 2. Part Location data

In [97]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [98]:
df_loc = pd.read_csv("Geospatial_Coordinates.csv")
df_loc.head(5)
del df_loc['Postal Code']
df_loc

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476
5,43.744734,-79.239476
6,43.727929,-79.262029
7,43.711112,-79.284577
8,43.716316,-79.239476
9,43.692657,-79.264848


## result data frame with Neighborhood and location 

In [102]:


result = pd.concat([df_toronto, df_loc], axis=1)
result = result.dropna(axis=0, how='any')
result

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M9A,Etobicoke,Islington Avenue,43.784535,-79.160497
2,M4A,North York,Victoria Village,43.763573,-79.188711
3,M4E,East Toronto,The Beaches,43.770992,-79.216917
4,M4K,East Toronto,Riverdale,43.773136,-79.239476
5,M1R,Scarborough,"Maryvale, Wexford",43.744734,-79.239476
6,M6S,West Toronto,"Runnymede, Swansea",43.727929,-79.262029
7,M8W,Etobicoke,"Alderwood, Long Branch",43.711112,-79.284577
8,M4Y,Downtown Toronto,Church and Wellesley,43.716316,-79.239476
9,M9W,Etobicoke,Northwest,43.692657,-79.264848


In [109]:
result.shape

(84, 5)

# 3. Part Explore Toronto

In [105]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   3.69 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  22.24 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  29.18 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  33.10 MB/s


## Map of Toronto

In [107]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

In [108]:
CLIENT_ID = 'g' # your Foursquare ID
CLIENT_SECRET = 'g' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [110]:
scarborough_data = result[result['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(7)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1R,Scarborough,"Maryvale, Wexford",43.744734,-79.239476
1,M1N,Scarborough,Birch Cliff,43.750072,-79.295849
2,M1W,Scarborough,Steeles West,43.794200,-79.262029
3,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.778517,-79.346556
4,M1J,Scarborough,Scarborough Village,43.782736,-79.442259
5,M1B,Scarborough,"Rouge, Malvern",43.728496,-79.495697
6,M1S,Scarborough,Agincourt,43.695344,-79.318389


In [113]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774


In [112]:
map_scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb

## top 100 venues in the neighborhood 'Steeles West', from Scarboroug

In [114]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighbourhood name


In [115]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [116]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c4a09b5db04f57d7516ba9d'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-5085ec39e4b0b1ead2eb0818-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/toys_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1f3941735',
         'name': 'Toy / Game Store',
         'pluralName': 'Toy / Game Stores',
         'primary': True,
         'shortName': 'Toys & Games'}],
       'id': '5085ec39e4b0b1ead2eb0818',
       'location': {'address': '300 Borough Drive',
        'cc': 'CA',
        'city': 'Scarborough',
        'country': 'Canada',
        'crossStreet': 'in Scarborough Town Centre',
        'distance': 284,
        'formattedAddress': ['300 Borough Drive (in Scarborough Town Centre)',
         'Scarborough ON M1P 4P5

In [117]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [118]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
2,DAVIDsTEA,Tea Room,43.776613,-79.258516
3,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
4,SEPHORA,Cosmetics Shop,43.775592,-79.258242
5,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
6,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
7,Scarborough Town Centre,Shopping Mall,43.775545,-79.257545
8,Moxie's Classic Grill,American Restaurant,43.776219,-79.256820
9,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645


In [119]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Get venues for each neighborhood in Scarborough

In [120]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Maryvale, Wexford
Birch Cliff
Steeles West
Clairlea, Golden Mile, Oakridge
Scarborough Village
Rouge, Malvern
Agincourt
Ionview, Kennedy Park
Cedarbrae
Tam O'Shanter
Upper Rouge
Cliffcrest, Cliffside
Morningside, West Hill
Highland Creek, Rouge Hill, Port Union
Woburn
Dorset Park, Scarborough Town Centre, Wexford Heights
Agincourt North, Milliken


In [121]:
scarborough_venues.head(10)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Maryvale, Wexford",43.744734,-79.239476,McCowan Park,43.745089,-79.239336,Playground
1,"Maryvale, Wexford",43.744734,-79.239476,Tumbe Cafe,43.744058,-79.244021,Grocery Store
2,Birch Cliff,43.750072,-79.295849,Wexford Restaurant,43.746030,-79.293843,Breakfast Spot
3,Birch Cliff,43.750072,-79.295849,Subway,43.746008,-79.293238,Sandwich Place
4,Birch Cliff,43.750072,-79.295849,Lebanese bakery,43.746701,-79.292896,Middle Eastern Restaurant
5,Birch Cliff,43.750072,-79.295849,Wexford Heights Plaza,43.746136,-79.293782,Shopping Mall
6,Birch Cliff,43.750072,-79.295849,Scarborough Garage Door Repair,43.751288,-79.301508,Auto Garage
7,Birch Cliff,43.750072,-79.295849,Puffin Gear,43.750947,-79.290047,Accessories Store
8,Birch Cliff,43.750072,-79.295849,Crown Pastries,43.746098,-79.293142,Bakery
9,Steeles West,43.794200,-79.262029,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot


In [122]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,9,9,9,9,9,9
"Agincourt North, Milliken",20,20,20,20,20,20
Birch Cliff,7,7,7,7,7,7
Cedarbrae,42,42,42,42,42,42
"Clairlea, Golden Mile, Oakridge",61,61,61,61,61,61
"Cliffcrest, Cliffside",100,100,100,100,100,100
"Dorset Park, Scarborough Town Centre, Wexford Heights",65,65,65,65,65,65
"Highland Creek, Rouge Hill, Port Union",4,4,4,4,4,4
"Ionview, Kennedy Park",4,4,4,4,4,4


In [123]:
# one hot encoding
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bakery,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [124]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped.head(7)

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.111111,0.111111,0.000000,0.0,0.0,...,0.000000,0.00000,0.00,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000
1,"Agincourt North, Milliken",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Birch Cliff,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.142857,0.0,0.0,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Cedarbrae,0.023810,0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.02381,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.000000,0.000000,0.016393,0.000000,0.032787,0.000000,0.000000,0.0,0.0,...,0.032787,0.00000,0.00,0.000000,0.016393,0.000000,0.000000,0.000000,0.016393,0.016393
5,"Cliffcrest, Cliffside",0.000000,0.000000,0.040000,0.010000,0.010000,0.000000,0.000000,0.0,0.0,...,0.000000,0.00000,0.01,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
6,"Dorset Park, Scarborough Town Centre, Wexford ...",0.015385,0.000000,0.015385,0.015385,0.046154,0.000000,0.000000,0.0,0.0,...,0.000000,0.00000,0.00,0.015385,0.000000,0.000000,0.030769,0.015385,0.000000,0.000000


 # Get top 10 venues per neighborhood

In [126]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [127]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Pharmacy,Park,Video Store,Asian Restaurant,Athletics & Sports,Cosmetics Shop,Beer Store,Skating Rink,Curling Ice,Department Store
1,"Agincourt North, Milliken",Breakfast Spot,Coffee Shop,Café,Stadium,Performing Arts Venue,Pet Store,Nightclub,Gym / Fitness Center,Climbing Gym,Caribbean Restaurant
2,Birch Cliff,Bakery,Auto Garage,Middle Eastern Restaurant,Shopping Mall,Sandwich Place,Breakfast Spot,Accessories Store,Cuban Restaurant,Coworking Space,Cupcake Shop
3,Cedarbrae,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Cosmetics Shop,Bubble Tea Shop,Caribbean Restaurant,Restaurant
4,"Clairlea, Golden Mile, Oakridge",Clothing Store,Fast Food Restaurant,Coffee Shop,Food Court,Restaurant,Bakery,Asian Restaurant,Toy / Game Store,Tea Room,Women's Store
5,"Cliffcrest, Cliffside",Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gym,Gastropub,Italian Restaurant,Deli / Bodega,Sports Bar
6,"Dorset Park, Scarborough Town Centre, Wexford ...",Bar,Men's Store,Restaurant,Asian Restaurant,Café,Coffee Shop,Pizza Place,Cocktail Bar,Bakery,Vietnamese Restaurant
7,"Highland Creek, Rouge Hill, Port Union",Pizza Place,Japanese Restaurant,Pub,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coffee Shop,Coworking Space,Electronics Store
8,"Ionview, Kennedy Park",Pub,Coffee Shop,Music Venue,Dog Run,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega,Curling Ice
9,"Maryvale, Wexford",Playground,Grocery Store,Curling Ice,Dog Run,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega,Women's Store


In [128]:
neighborhoods_venues_sorted.shape

(17, 11)

## Run k-means to cluster the neighborhoods into 5 clusters

In [144]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

scarb_data = scarborough_data
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
len(kmeans.labels_)#=16
#scarborough_data.shape

17

## Include kmeans.labels_ into the original Scarborough dataframe

In [145]:
scarb_merged = scarb_data

# add clustering labels
scarb_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarb_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1R,Scarborough,"Maryvale, Wexford",43.744734,-79.239476,0,Playground,Grocery Store,Curling Ice,Dog Run,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega,Women's Store
1,M1N,Scarborough,Birch Cliff,43.750072,-79.295849,0,Bakery,Auto Garage,Middle Eastern Restaurant,Shopping Mall,Sandwich Place,Breakfast Spot,Accessories Store,Cuban Restaurant,Coworking Space,Cupcake Shop
2,M1W,Scarborough,Steeles West,43.794200,-79.262029,0,Breakfast Spot,Clothing Store,Lounge,Skating Rink,Department Store,Electronics Store,Dog Run,Diner,Dim Sum Restaurant,Dessert Shop
3,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.778517,-79.346556,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Food Court,Restaurant,Bakery,Asian Restaurant,Toy / Game Store,Tea Room,Women's Store
4,M1J,Scarborough,Scarborough Village,43.782736,-79.442259,0,Coffee Shop,Grocery Store,Butcher,Pizza Place,Pharmacy,Greek Restaurant,Diner,Comfort Food Restaurant,Concert Hall,Convenience Store
5,M1B,Scarborough,"Rouge, Malvern",43.728496,-79.495697,0,Home Service,Food Truck,Baseball Field,Department Store,Electronics Store,Dog Run,Diner,Dim Sum Restaurant,Dessert Shop,Deli / Bodega
6,M1S,Scarborough,Agincourt,43.695344,-79.318389,0,Pharmacy,Park,Video Store,Asian Restaurant,Athletics & Sports,Cosmetics Shop,Beer Store,Skating Rink,Curling Ice,Department Store
7,M1K,Scarborough,"Ionview, Kennedy Park",43.676357,-79.293031,2,Pub,Coffee Shop,Music Venue,Dog Run,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega,Curling Ice
8,M1H,Scarborough,Cedarbrae,43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Cosmetics Shop,Bubble Tea Shop,Caribbean Restaurant,Restaurant
9,M1T,Scarborough,Tam O'Shanter,43.659526,-79.340923,3,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Yoga Studio,Music Store,Cheese Shop,Chinese Restaurant,Pizza Place


# Map

In [146]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighbourhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

 # Examine each of the five clusters


In [147]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Playground,Grocery Store,Curling Ice,Dog Run,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega,Women's Store
1,Scarborough,0,Bakery,Auto Garage,Middle Eastern Restaurant,Shopping Mall,Sandwich Place,Breakfast Spot,Accessories Store,Cuban Restaurant,Coworking Space,Cupcake Shop
2,Scarborough,0,Breakfast Spot,Clothing Store,Lounge,Skating Rink,Department Store,Electronics Store,Dog Run,Diner,Dim Sum Restaurant,Dessert Shop
3,Scarborough,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Food Court,Restaurant,Bakery,Asian Restaurant,Toy / Game Store,Tea Room,Women's Store
4,Scarborough,0,Coffee Shop,Grocery Store,Butcher,Pizza Place,Pharmacy,Greek Restaurant,Diner,Comfort Food Restaurant,Concert Hall,Convenience Store
5,Scarborough,0,Home Service,Food Truck,Baseball Field,Department Store,Electronics Store,Dog Run,Diner,Dim Sum Restaurant,Dessert Shop,Deli / Bodega
6,Scarborough,0,Pharmacy,Park,Video Store,Asian Restaurant,Athletics & Sports,Cosmetics Shop,Beer Store,Skating Rink,Curling Ice,Department Store
8,Scarborough,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Cosmetics Shop,Bubble Tea Shop,Caribbean Restaurant,Restaurant
10,Scarborough,0,Park,Dim Sum Restaurant,Swim School,Bus Line,Women's Store,Deli / Bodega,Dog Run,Diner,Dessert Shop,Department Store
12,Scarborough,0,Coffee Shop,Sandwich Place,Café,Pizza Place,Jewish Restaurant,Park,Cosmetics Shop,Pub,Martial Arts Dojo,Burger Joint


In [148]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,1,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gym,Gastropub,Italian Restaurant,Deli / Bodega,Sports Bar


In [149]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Scarborough,2,Pub,Coffee Shop,Music Venue,Dog Run,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega,Curling Ice


In [150]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,3,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Yoga Studio,Music Store,Cheese Shop,Chinese Restaurant,Pizza Place


In [151]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Scarborough,4,Pizza Place,Japanese Restaurant,Pub,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coffee Shop,Coworking Space,Electronics Store
